In [127]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import os.path
from os.path import join
import numpy as np
import imodelsx
from tqdm import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import data
import sys
files_dict = data.load_files_dict_single_site()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Kevin_AnnArbor_LosAngeles_SaltLakeCity
Jonathan_Indianapolis_Milwaukee_Pittsburgh


/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Katie_Dallas_Seattle_Tucson


/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Spencer_Denver_Sacramento
Patty_Charlotte_Houston_Portland
Corina_Buffalo_Cincinnati_SanFrancisco
Manish_Atlanta_Columbus_WashingtonDC
success: 0
fail: ['Los Angeles COMPLETE', 'Salt Lake City', 'Ann Arbor', 'Pittsburgh', 'Milwaukee', 'Indianapolis', 'Tucson', 'Dallas', 'Seattle', 'Denver', 'Sacramento', 'Houston', 'Charlotte', 'Portland', 'Cincinnati', 'San Francisco', 'Buffalo', 'Columbus', 'WashingtonDC', 'Atlanta']


/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


### Filter data for single-site analysis

In [ ]:
k = 'Atlanta'
# k = 'Columbus'
df = files_dict[k]
theme_index = np.where(
    np.array(list(map(str.lower, df.columns.values))) == 'theme')[0][0]
col_vals = df.columns[4: theme_index]

# separate into relevant pieces
qs = df['Subcategory']
responses_df = df[col_vals]
themes_df = df[df.columns[theme_index:]]

In [ ]:
def numbered_list(responses):
    return '\n'.join([f'{i+1}. {c.strip()}' for i, c in enumerate(responses)])


themes_prompt = '''### You are given a question and a set of responses below.

**Question**: {question}

**Responses**:
{response_list}

### Group all responses into 2 or more non-overlapping themes.
### Return a comma-separated list, where each element is a theme, followed by the numbers of the responses that fall into that theme in brackets.
### **Example answer**: Theme 1: Negative responses [1, 2, 5], Theme 2: Positive responses [3, 4]

**Answer**: Theme 1:'''

llm = imodelsx.llm.get_llm('gpt-4', repeat_delay=3)

**Run single example**

In [ ]:
# question, responses, theme_dict = data.get_data_for_question_single_site(
#     question_num=2, qs=qs, responses_df=responses_df, themes_df=themes_df)

# resps = responses[pd.notna(responses)]
# prompt = themes_prompt.format(
#     question=question.strip(),
#     response_list=numbered_list(resps)
# )
# print(prompt)
# llm(prompt)

### Screen valid questions
Valid questions have multiple unique responses.

In [ ]:
def count_unique(resps):
    resps_match = resps.apply(str.lower)
    resps_match = resps_match.str.replace('[^\w\s]', '')
    # print(set(resps_match))
    return len(set(resps_match))


# screen valid questions
valid_question_nums = []
for question_num in tqdm(range(len(qs)), position=0):

    question, responses, theme_dict = data.get_data_for_question_single_site(
        question_num=question_num, qs=qs, responses_df=responses_df, themes_df=themes_df)
    resps = responses[pd.notna(responses)]

    # valid only if there are multiple unique responses
    if count_unique(resps) > 3:
        valid_question_nums.append(question_num)

    # print(np.unique(resps))
print('num valid qs', len(valid_question_nums), 'of', len(qs))

### Run generating themes

In [ ]:
themes_generated = {}
resps_list = {}
for question_num in tqdm(valid_question_nums, position=0):

    question, responses, theme_dict = data.get_data_for_question_single_site(
        question_num=question_num, qs=qs, responses_df=responses_df, themes_df=themes_df)
    resps = responses[pd.notna(responses)]

    prompt = themes_prompt.format(
        question=question.strip(),
        response_list=numbered_list(resps)
    )
    ans = llm(prompt)
    themes_generated[question_num] = ans
    resps.index = np.arange(len(resps)) + 1
    resps_list[question_num] = resps

In [139]:
def dprint(*args, f):
    # print(*args, file=sys.stdout)
    print(*args, file=f)


with open('../figs/themes_generated.md', 'w') as f:
    for question_num in valid_question_nums:
        dprint('### Question:', qs[question_num], f=f)
        dprint('\nResponses', f=f)
        for i in range(len(resps_list[question_num])):
            dprint(f'{i+1}. {resps_list[question_num].iloc[i]}', f=f)
        dprint('\nThemes', f=f)
        # print(themes_generated[question_num])
        themes = [s.strip(' ,:1234567890')
                  for s in themes_generated[question_num].split('Theme')]
        for i, theme in enumerate(themes):
            dprint(f'- Theme {i + 1}:', theme, f=f)
        dprint('', f=f)

In [ ]:
resps_list